In [41]:
class Agent():
    def __init__(self,state,parent,action,depth,step_cost,path_cost,heuristic_cost):
        self.state = state 
        self.parent = parent # parent Node
        self.action = action # move up, left, down, right
        self.depth = depth # depth of the Node in the tree
        self.step_cost = step_cost # g(n), the cost to take the step
        self.path_cost = path_cost # accumulated g(n), the cost to reach the current Node
        self.heuristic_cost = heuristic_cost # h(n), cost to reach goal state from the current Node
        
        # children  Node
        self.move_up = None 
        self.move_left = None
        self.move_down = None
        self.move_right = None

    # once the goal Node is found, track back to the root Node and print out the path
    def print_path(self):
        # create FILO stacks to place the track
        action_track = [self.action]
        depth_track = [self.depth]
        state_track = [self.state]
        step_cost_track = [self.step_cost]
        path_cost_track = [self.path_cost]
        heuristic_cost_track = [self.heuristic_cost]

        # add Agent information as tracing back up the tree
        while self.parent:
            self = self.parent
            action_track.append(self.action) #the action that was taken to reach the node
            depth_track.append(self.depth) #the depth of the node in the search tree
            state_track.append(self.state) #the state of the node
            step_cost_track.append(self.step_cost) # the cost of taking the action to reach the node
            path_cost_track.append(self.path_cost)  #the total cost of reaching the node, which is the sum of the path cost and the heuristic cost
            heuristic_cost_track.append(self.heuristic_cost)

        # print out the path
        step_counter = 0
        while state_track:
            print ('step',step_counter)
            print (state_track.pop())
            print ('action=',action_track.pop(),', depth=',str(depth_track.pop()),', step cost=',str(step_cost_track.pop()),', total_cost=',
            str(path_cost_track.pop() + heuristic_cost_track.pop()),'\n')
            
            step_counter += 1


    # see if moving up is valid
    def moving_up(self):
        index_0=[i[0] for i in np.where(self.state==0)] #searches the index value of 0 inthe current state
        if index_0[0] == 2: #if 0 value is already in the top row cannot move futher
            return None
        else:
            lower_value = self.state[index_0[0]+1,index_0[1]] # If not in the top row, then it can move up one row. get the value of the tile that the 0 value will swap with.
            current_state = self.state.copy()  #create copy of the new state
            current_state[index_0[0],index_0[1]] = lower_value  #this line swaps the 0 value with the tile that is below it (index position of 0 in matrix)
            current_state[index_0[0]+1,index_0[1]] = 0   #This line sets the tile that the 0 value was swapped with to be 0.
            return current_state,lower_value  #This line returns a tuple containing the new puzzle state after moving the 0 value up and the value of the tile that the 0 value was swapped with

    # see if moving down is valid
    def moving_down(self):
        # index of the empty tile
        index_0=[i[0] for i in np.where(self.state==0)] 
        if index_0[0] == 0:
            return None
        else:
            up_value = self.state[index_0[0]-1,index_0[1]] # value of the upper tile
            current_state = self.state.copy()
            current_state[index_0[0],index_0[1]] = up_value
            current_state[index_0[0]-1,index_0[1]] = 0
            return current_state,up_value

    # see if moving left is valid
    def moving_left(self):
        index_0=[i[0] for i in np.where(self.state==0)] 
        if index_0[1] == 2:
            return None
        else:
            right_value = self.state[index_0[0],index_0[1]+1] # value of the right tile
            current_state = self.state.copy()
            current_state[index_0[0],index_0[1]] = right_value
            current_state[index_0[0],index_0[1]+1] = 0
            return current_state,right_value

    # see if moving right is valid
    def moving_right(self):
        index_0=[i[0] for i in np.where(self.state==0)] 
        if index_0[1] == 0:
            return None
        else:
            left_value = self.state[index_0[0],index_0[1]-1] # value of the left tile
            current_state = self.state.copy()
            current_state[index_0[0],index_0[1]] = left_value
            current_state[index_0[0],index_0[1]-1] = 0
            return current_state,left_value
        
    # return user specified heuristic cost
    def get_h_cost(self,current_state,goal_state,heuristic_function,path_cost,depth):
        if heuristic_function == 'manhattan': #depending on the heuristic function calculates the heuristic cost by manhattan distance
            return self.h_manhattan_cost(current_state,goal_state) #will call manhattan function to calculate the cost between each tile and its correct position
        # In the original game the step cost is the number on the tile
        # I have come up with the relaxed version of the problem and, I made all the step cost as 1
        # made it a Greedy search with manhattan heuristic function
        elif heuristic_function == 'fair_manhattan':
            return self.h_manhattan_cost(current_state,goal_state) - path_cost + depth

    # return heuristic cost: sum of Manhattan distance to reach the goal state
    def h_manhattan_cost(self,current_state,goal_state): #the sum of the Manhattan distances between each tile and its correct position.
        current = current_state
        # digit and coordinates they are supposed to be
        goal_position = {1:(0,0),2:(0,1),3:(0,2),8:(1,0),0:(1,1),4:(1,2),7:(2,0),6:(2,1),5:(2,2)} 
        man_sum = 0
        for i in range(3):# for row in current position
            for j in range(3): # for column
                if current[i,j] != 0:
                    man_sum += sum(abs(a-b) for a,b in zip((i,j), goal_position[current[i,j]])) #the zip fun. pairs up two tuples which contains the current position and goal position of a tile the list comprehension of the distance between them
        return man_sum
        

#BFS   (First in First out)       

    def breadth_first_search(self, goal_state):
        fringe = [self] # queue of found but unvisited_nodes Nodes, (root node)
        fringe_n_popped = 0 # number of Nodes popped off the fringe
        fringe_max_length = 1 # max number of Nodes in the fringe
        fringe_d = [0] #  fringe of Node depth
        path_cost_fringe = [0] # nodes in the fringe for path cost
        visited_nodes = set([]) # record visited_nodes states
        
        # loop for each iteration 
        while fringe:
            # update maximum length of the fringe
            if len(fringe) > fringe_max_length:
                fringe_max_length = len(fringe)
                
            current_d = fringe_d.pop(0) # select and remove the depth for current node
            current_n = fringe.pop(0) # select and remove the first node in the fringe
            fringe_n_popped += 1
            current_p_cost = path_cost_fringe.pop(0) # select and remove the path cost for reaching current node
            visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # saves the visited_nodes nodes in the tuple to avoid repeatition 
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(current_n.state,goal_state):
                current_n.print_path()
                print ('Nodes Poped',str(fringe_n_popped))
                print ('Max Fringe Size', str(fringe_max_length))
                return 'Search Complete'      
            else:                
                # see if moving lower tile up is a valid move
                if current_n.moving_up():
                    current_state,lower_value = current_n.moving_up()
                    # check if the resulting node is already visited_nodes to avoid repeatation
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=current_d+1,\
                                              step_cost=lower_value,path_cost=current_p_cost+lower_value,heuristic_cost=0)
                        fringe.append(current_n.move_up) #we add the node to the end of the fringe with append
                        fringe_d.append(current_d+1)
                        path_cost_fringe.append(current_p_cost+lower_value)

                # see if moving upper tile down is a valid move
                if current_n.moving_down():
                    current_state,up_value = current_n.moving_down()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=current_d+1,\
                                              step_cost=up_value,path_cost=current_p_cost+up_value,heuristic_cost=0)
                        fringe.append(current_n.move_down) #we add to the end of the fringe by append since it is BFS
                        fringe_d.append(current_d+1)
                        path_cost_fringe.append(current_p_cost+up_value)

                # see if moving right tile to the left is a valid move
                if current_n.moving_left():
                    current_state,right_value = current_n.moving_left()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=current_d+1,\
                                              step_cost=right_value,path_cost=current_p_cost+right_value,heuristic_cost=0)
                        fringe.append(current_n.move_left)
                        fringe_d.append(current_d+1)
                        path_cost_fringe.append(current_p_cost+right_value)
                
               # see if moving left tile to the right is a valid move
                if current_n.moving_right():
                    current_state,left_value = current_n.moving_right()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes: 
                        # create a new child node
                        current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=current_d+1,\
                                              step_cost=left_value,path_cost=current_p_cost+left_value,heuristic_cost=0)
                        fringe.append(current_n.move_right) 
                        fringe_d.append(current_d+1)
                        path_cost_fringe.append(current_p_cost+left_value)


#UCS (cost of all the actions to get to root node), An extension of BFS that's guided by a prioritized fringe based on path cost

    def uniform_cost_search_BFS(self, goal_state):
        fringe = [(self,0)] # fringe of (found but unvisited_nodes, path cost), ordered by path cost(accumulated step cost)
        fringe_n_popped = 0 # number of nodes popped off the fringe
        fringe_max_length = 1 # max number of nodes in the fringe
        path_cost_fringe = [0] # fringe for path cost
        fringe_d = [(0,0)] # fringe of node depth, (depth, path cost)
        visited_nodes = set([]) # record visited_nodes
        # loop for each iteration 
        while fringe:
            # sort fringe based on path cost, in ascending order so it pops the node with least cost first from the fringe
            fringe = sorted(fringe, key=lambda x: x[1])
            fringe_d = sorted(fringe_d, key=lambda x: x[1])
            path_cost_fringe = sorted(path_cost_fringe, key=lambda x: x)
            
            # update maximum length of the fringe
            if len(fringe) > fringe_max_length:
                fringe_max_length = len(fringe)
                
            current_n = fringe.pop(0)[0] # select and remove the first node in the fringe
            fringe_n_popped += 1
            current_p_cost = path_cost_fringe.pop(0) #  select and remove the path cost for reaching current node
            current_d = fringe_d.pop(0)[0] # select and remove the depth for current node
            visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(current_n.state,goal_state):
                current_n.print_path()
                print ('Nodes Poped',str(fringe_n_popped))
                print ('Max Fringe Size', str(fringe_max_length))
                return 'Search Completed'
            
            else:     

                # see if moving lower tile up is a valid move
                if current_n.moving_up():
                    current_state,lower_value = current_n.moving_up()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=current_d+1,\
                                              step_cost=lower_value,path_cost=current_p_cost+lower_value,heuristic_cost=0)
                        fringe.append((current_n.move_up,current_p_cost+lower_value)) # added back of the fringe but gets sorted
                        fringe_d.append((current_d+1,current_p_cost+lower_value))
                        path_cost_fringe.append(current_p_cost+lower_value)

                # see if moving upper tile down is a valid move
                if current_n.moving_down():
                    current_state,up_value = current_n.moving_down()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=current_d+1,\
                                              step_cost=up_value,path_cost=current_p_cost+up_value,heuristic_cost=0)
                        fringe.append((current_n.move_down,current_p_cost+up_value))
                        fringe_d.append((current_d+1,current_p_cost+up_value))
                        path_cost_fringe.append(current_p_cost+up_value)

                # see if moving right tile to the left is a valid move
                if current_n.moving_left():
                    current_state,right_value = current_n.moving_left()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=current_d+1,\
                                              step_cost=right_value,path_cost=current_p_cost+right_value,heuristic_cost=0)
                        fringe.append((current_n.move_left,current_p_cost+right_value))
                        fringe_d.append((current_d+1,current_p_cost+right_value))
                        path_cost_fringe.append(current_p_cost+right_value)

                # see if moving left tile to the right is a valid move
                if current_n.moving_right():
                    current_state,left_value = current_n.moving_right()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=current_d+1,\
                                              step_cost=left_value,path_cost=current_p_cost+left_value,heuristic_cost=0)
                        fringe.append((current_n.move_right,current_p_cost+left_value))
                        fringe_d.append((current_d+1,current_p_cost+left_value))
                        path_cost_fringe.append(current_p_cost+left_value)


#DFS (Last in First Out)(will find the shallowest solution)
          
    def depth_first_search(self, goal_state):     
        fringe = [self] # fringe of found but unvisited_nodes
        fringe_n_popped = 0 # number of nodes popped off the fringe
        fringe_max_length = 1 # max number of nodes in the fringe
        path_cost_fringe = [0] # fringe for path cost
        fringe_d = [0] # fringe of node depth
        visited_nodes = set([]) # record visited_nodes states
        # loop for each iteration 
        while fringe:
            # update maximum length of the fringe
            if len(fringe) > fringe_max_length:
                fringe_max_length = len(fringe)
            current_n = fringe.pop(0) # select and remove the first Agent in the fringe
            fringe_n_popped += 1 
            current_d = fringe_d.pop(0) # select and remove the depth for current node
            current_p_cost = path_cost_fringe.pop(0) # # select and remove the depth for current no# select and remove the path cost for reaching current node
            visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(current_n.state,goal_state):
                current_n.print_path()
                print ('Nodes Poped',str(fringe_n_popped))
                print ('Max Fringe Size', str(fringe_max_length))
                return 'Search Complete'        
            else:      
                # see if moving lower tile up is a valid move
                if current_n.moving_up():
                    current_state,lower_value = current_n.moving_up()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=current_d+1,\
                                              step_cost=lower_value,path_cost=current_p_cost+lower_value,heuristic_cost=0)
                        fringe.insert(0,current_n.move_up) # inserted to the begining of the fringe so it pops the first
                        fringe_d.insert(0,current_d+1)
                        path_cost_fringe.insert(0,current_p_cost+lower_value)

                # see if moving upper tile down is a valid move
                if current_n.moving_down():
                    current_state,up_value = current_n.moving_down()
                    # check if the resulting Agent is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child Agent
                        current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=current_d+1,\
                                              step_cost=up_value,path_cost=current_p_cost+up_value,heuristic_cost=0)
                        fringe.insert(0,current_n.move_down) # inserted to the begining of the fringe
                        fringe_d.insert(0,current_d+1)
                        path_cost_fringe.insert(0,current_p_cost+up_value)

                # see if moving right tile to the left is a valid move
                if current_n.moving_left():
                    current_state,right_value = current_n.moving_left()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=current_d+1,\
                                              step_cost=right_value,path_cost=current_p_cost+right_value,heuristic_cost=0)
                        fringe.insert(0,current_n.move_left)
                        fringe_d.insert(0,current_d+1)
                        path_cost_fringe.insert(0,current_p_cost+right_value)

                # see if moving left tile to the right is a valid move
                if current_n.moving_right():
                    current_state,left_value = current_n.moving_right()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # create a new child node
                        current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=current_d+1,\
                                              step_cost=left_value,path_cost=current_p_cost+left_value,heuristic_cost=0)
                        fringe.insert(0,current_n.move_right)
                        fringe_d.insert(0,current_d+1)
                        path_cost_fringe.insert(0,current_p_cost+left_value)

         

#IDS, An extension of DFS that's guided by LIFO (we set the depth limit as an input) enter value greater than 14

    def iterative_deepening_DFS(self, goal_state):
        fringe_n_popped = 0 # number of nodes popped off the fringe
        fringe_max_length = 1 # max number of nodes in the fringe
        # search the tree that's 40 levels in depth
        for depth_limit in range(int(input('what is the dept limit: '))): #enter limit >=14
            fringe = [self] # fringe of found but unvisited_nodes nodes, FILO
            fringe_d = [0] # fringe of node depth
            path_cost_fringe = [0] # fringe for path cost
            visited_nodes = set([]) # record visited_nodes states
            while fringe:
                # update maximum length of the fringe
                if len(fringe) > fringe_max_length:
                    fringe_max_length = len(fringe)

                current_n = fringe.pop(0) # select and remove the first node in the fringe
                fringe_n_popped += 1 
                current_d = fringe_d.pop(0) # select and remove the depth for current node
                current_p_cost = path_cost_fringe.pop(0) # select and remove the path cost for reaching current node
                visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # add state, which is represented as a tuple

                # when the goal state is found, track back to the root node and print out the path
                if np.array_equal(current_n.state,goal_state):
                    current_n.print_path()
                    print ('Nodes Poped',str(fringe_n_popped))
                    print ('Max Fringe Size', str(fringe_max_length))
                    return 'Search Complete'

                else:              
                    if current_d < depth_limit:

                        # see if moving lower tile up is a valid move
                        if current_n.moving_up():
                            current_state,lower_value = current_n.moving_up()
                            # check if the resulting Agent is already visited_nodes
                            if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                                # create a new child Agent
                                current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=current_d+1,\
                                                      step_cost=lower_value,path_cost=current_p_cost+lower_value,heuristic_cost=0)
                                fringe.insert(0,current_n.move_up) #insert at the start of the fringe
                                fringe_d.insert(0,current_d+1)
                                path_cost_fringe.insert(0,current_p_cost+lower_value)

                        # see if moving upper tile down is a valid move
                        if current_n.moving_down():
                            current_state,up_value = current_n.moving_down()
                            # check if the resulting Agent is already visited_nodes
                            if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                                # create a new child Agent
                                current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=current_d+1,\
                                                      step_cost=up_value,path_cost=current_p_cost+up_value,heuristic_cost=0)
                                fringe.insert(0,current_n.move_down) # inserted to the begining of the fringe
                                fringe_d.insert(0,current_d+1)
                                path_cost_fringe.insert(0,current_p_cost+up_value)

                       # see if moving right tile to the left is a valid move
                        if current_n.moving_left():
                            current_state,right_value = current_n.moving_left()
                            # check if the resulting Agent is already visited_nodes
                            if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                                # create a new child Agent
                                current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=current_d+1,\
                                                      step_cost=right_value,path_cost=current_p_cost+right_value,heuristic_cost=0)
                                fringe.insert(0,current_n.move_left)
                                fringe_d.insert(0,current_d+1)
                                path_cost_fringe.insert(0,current_p_cost+right_value)

                        # see if moving left tile to the right is a valid move
                        if current_n.moving_right():
                            current_state,left_value = current_n.moving_right()
                            # check if the resulting Agent is already visited_nodes
                            if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                                # create a new child Agent
                                current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=current_d+1,\
                                                      step_cost=left_value,path_cost=current_p_cost+left_value,heuristic_cost=0)
                                fringe.insert(0,current_n.move_right)
                                fringe_d.insert(0,current_d+1)
                                path_cost_fringe.insert(0,current_p_cost+left_value)
                    

 #Greedy Search                       
    # search based on heuristic cost (manhattan distance)
    def greedy_search(self, goal_state):
        fringe = [(self,0)] # fringe of (found but unvisited_nodes, heuristic cost), ordered by heuristic cost
        fringe_n_popped = 0 # number of node popped off the fringe
        path_cost_fringe = [(0,0)] # fringe for path cost, (path_cost, heuristic cost)
        fringe_max_length = 1 # max number of node in the fringe
        fringe_d = [(0,0)] # fringe of node depth, (depth, heuristic cost)
        visited_nodes = set([]) # record visited_nodes states
        
        while fringe:
            # sort fringe based on heuristic cost, in ascending order
            fringe = sorted(fringe, key=lambda x: x[1])
            fringe_d = sorted(fringe_d, key=lambda x: x[1])
            path_cost_fringe = sorted(path_cost_fringe, key=lambda x: x[1])
            # update maximum length of the fringe
            if len(fringe) > fringe_max_length:
                fringe_max_length = len(fringe)
            current_n = fringe.pop(0)[0] # select and remove the first node in the fringe
            fringe_n_popped += 1 
            current_d = fringe_d.pop(0)[0] # select and remove the depth for current node
            current_p_cost = path_cost_fringe.pop(0)[0] # select and remove the path cost for reaching current node
            visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(current_n.state,goal_state):
                current_n.print_path()
                print ('Agents Poped',str(fringe_n_popped))
                print ('Max Fringe Size', str(fringe_max_length))
                return 'Search Complete'
            else:     
                # see if moving lower tile up is a valid move
                if current_n.moving_up():
                    current_state,lower_value = current_n.moving_up()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # get heuristic cost
                        h_cost = self.h_manhattan_cost(current_state,goal_state)
                        # create a new child nodes
                        current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=current_d+1,\
                                              step_cost=lower_value,path_cost=current_p_cost+lower_value,heuristic_cost=h_cost)
                        fringe.append((current_n.move_up,h_cost))
                        fringe_d.append((current_d+1,h_cost))
                        path_cost_fringe.append((current_p_cost+lower_value,h_cost))

                # see if moving upper tile down is a valid move
                if current_n.moving_down():
                    current_state,up_value = current_n.moving_down()
                    # check if the resulting mode is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # get heuristic cost
                        h_cost = self.h_manhattan_cost(current_state,goal_state)
                        # create a new child nodes
                        current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=current_d+1,\
                                              step_cost=up_value,path_cost=current_p_cost+up_value,heuristic_cost=h_cost)
                        fringe.append((current_n.move_down,h_cost))
                        fringe_d.append((current_d+1,h_cost))
                        path_cost_fringe.append((current_p_cost+up_value,h_cost))
               
               # see if moving right tile to the left is a valid move
                if current_n.moving_left():
                    current_state,right_value = current_n.moving_left()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # get heuristic cost
                        h_cost = self.h_manhattan_cost(current_state,goal_state)
                        # create a new child nodes
                        current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=current_d+1,\
                                              step_cost=right_value,path_cost=current_p_cost+right_value,heuristic_cost=h_cost)
                        fringe.append((current_n.move_left,h_cost))
                        fringe_d.append((current_d+1,h_cost))
                        path_cost_fringe.append((current_p_cost+right_value,h_cost))

                # see if moving left tile to the right is a valid move
                if current_n.moving_right():
                    current_state,left_value = current_n.moving_right()
                    # check if the resulting nodes is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        # get heuristic cost
                        h_cost = self.h_manhattan_cost(current_state,goal_state)
                        # create a new child nodes
                        current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=current_d+1,\
                                              step_cost=left_value,path_cost=current_p_cost+left_value,heuristic_cost=h_cost)
                        fringe.append((current_n.move_right,h_cost))
                        fringe_d.append((current_d+1,h_cost))
                        path_cost_fringe.append((current_p_cost+left_value,h_cost))

#A* Search (g(n)+h(n)) search based on path cost + heuristic cost

    def a_star_search(self,goal_state,heuristic_function):
        fringe = [(self,0)] # fringe of (found but unvisited_nodes, path cost+heuristic cost), ordered by the second element
        fringe_n_popped = 0 # number of Agents popped off the fringe, measuring time performance
        fringe_max_length = 1 # max number of Agents in the fringe, measuring space performance
        fringe_d = [(0,0)] # fringe of node depth, (depth, path_cost+heuristic cost)
        path_cost_fringe = [(0,0)] # fringe for path cost, (path_cost, path_cost+heuristic cost)
        visited_nodes = set([]) # record visited_nodes states
        
        while fringe:
            # sort fringe based on path_cost+heuristic cost, in ascending order
            fringe = sorted(fringe, key=lambda x: x[1])
            fringe_d = sorted(fringe_d, key=lambda x: x[1])
            path_cost_fringe = sorted(path_cost_fringe, key=lambda x: x[1])
            # update maximum length of the fringe
            if len(fringe) > fringe_max_length:
                fringe_max_length = len(fringe)
                
            current_n = fringe.pop(0)[0] # select and remove the first node in the fringe
            fringe_n_popped += 1 
            current_d = fringe_d.pop(0)[0] # select and remove the depth for current node
            current_p_cost = path_cost_fringe.pop(0)[0] # select and remove the path cost for reaching current node
            visited_nodes.add(tuple(current_n.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(current_n.state,goal_state):
                current_n.print_path()
                print ('Agents Poped',str(fringe_n_popped))
                print ('Max Fringe Size', str(fringe_max_length))
                return 'Search Complete'
            else:     

                # see if moving lower tile up is a valid move
                if current_n.moving_up():
                    current_state,lower_value = current_n.moving_up()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        path_cost=current_p_cost+lower_value
                        depth = current_d+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(current_state,goal_state,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost
                        current_n.move_up = Agent(state=current_state,parent=current_n,action='up',depth=depth,\
                                              step_cost=lower_value,path_cost=path_cost,heuristic_cost=h_cost)
                        fringe.append((current_n.move_up, total_cost))
                        fringe_d.append((depth, total_cost))
                        path_cost_fringe.append((path_cost, total_cost))

                # see if moving upper tile down is a valid move
                if current_n.moving_down():
                    current_state,up_value = current_n.moving_down()
                    # check if the resulting Agent is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        path_cost=current_p_cost+up_value
                        depth = current_d+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(current_state,goal_state,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost
                        current_n.move_down = Agent(state=current_state,parent=current_n,action='down',depth=depth,\
                                              step_cost=up_value,path_cost=path_cost,heuristic_cost=h_cost)
                        fringe.append((current_n.move_down, total_cost))
                        fringe_d.append((depth, total_cost))
                        path_cost_fringe.append((path_cost, total_cost))
                    
                # see if moving right tile to the left is a valid move
                if current_n.moving_left():
                    current_state,right_value = current_n.moving_left()
                    # check if the resulting Agent is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        path_cost=current_p_cost+right_value
                        depth = current_d+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(current_state,goal_state,heuristic_function,path_cost,depth)
                        # create a new child Agent
                        total_cost = path_cost+h_cost
                        current_n.move_left = Agent(state=current_state,parent=current_n,action='left',depth=depth,\
                                              step_cost=right_value,path_cost=path_cost,heuristic_cost=h_cost)
                        fringe.append((current_n.move_left, total_cost))
                        fringe_d.append((depth, total_cost))
                        path_cost_fringe.append((path_cost, total_cost))

                # see if moving left tile to the right is a valid move
                if current_n.moving_right():
                    current_state,left_value = current_n.moving_right()
                    # check if the resulting node is already visited_nodes
                    if tuple(current_state.reshape(1,9)[0]) not in visited_nodes:
                        path_cost=current_p_cost+left_value
                        depth = current_d+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(current_state,goal_state,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost
                        current_n.move_right = Agent(state=current_state,parent=current_n,action='right',depth=depth,\
                                              step_cost=left_value,path_cost=path_cost,heuristic_cost=h_cost)
                        fringe.append((current_n.move_right, total_cost))
                        fringe_d.append((depth, total_cost))
                        path_cost_fringe.append((path_cost, total_cost))


In [42]:
import numpy as np
start_state= np.array([2,3,6,1,0,7,4,8,5]).reshape(3,3)
goal_state = np.array([1,2,3,4,5,6,7,8,0]).reshape(3,3)
print (start_state,'\n')
print (goal_state)

[[2 3 6]
 [1 0 7]
 [4 8 5]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [43]:
# calling the function 
root_node = Agent(state=start_state,parent=None,action=None,depth=0,step_cost=0,path_cost=0,heuristic_cost=0)

In [44]:
# search level by level with queue
root_node.breadth_first_search(goal_state)

step 0
[[2 3 6]
 [1 0 7]
 [4 8 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 3 6]
 [1 7 0]
 [4 8 5]]
action= left , depth= 1 , step cost= 7 , total_cost= 7 

step 2
[[2 3 6]
 [1 7 5]
 [4 8 0]]
action= up , depth= 2 , step cost= 5 , total_cost= 12 

step 3
[[2 3 6]
 [1 7 5]
 [4 0 8]]
action= right , depth= 3 , step cost= 8 , total_cost= 20 

step 4
[[2 3 6]
 [1 0 5]
 [4 7 8]]
action= down , depth= 4 , step cost= 7 , total_cost= 27 

step 5
[[2 3 6]
 [1 5 0]
 [4 7 8]]
action= left , depth= 5 , step cost= 5 , total_cost= 32 

step 6
[[2 3 0]
 [1 5 6]
 [4 7 8]]
action= down , depth= 6 , step cost= 6 , total_cost= 38 

step 7
[[2 0 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 7 , step cost= 3 , total_cost= 41 

step 8
[[0 2 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 8 , step cost= 2 , total_cost= 43 

step 9
[[1 2 3]
 [0 5 6]
 [4 7 8]]
action= up , depth= 9 , step cost= 1 , total_cost= 44 

step 10
[[1 2 3]
 [4 5 6]
 [0 7 8]]
action= up , depth= 10 , step cost= 4

'Search Complete'

In [45]:
# search based on path cost, using priority queue
root_node.uniform_cost_search_BFS(goal_state)

step 0
[[2 3 6]
 [1 0 7]
 [4 8 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 3 6]
 [1 7 0]
 [4 8 5]]
action= left , depth= 1 , step cost= 7 , total_cost= 7 

step 2
[[2 3 6]
 [1 7 5]
 [4 8 0]]
action= up , depth= 2 , step cost= 5 , total_cost= 12 

step 3
[[2 3 6]
 [1 7 5]
 [4 0 8]]
action= right , depth= 3 , step cost= 8 , total_cost= 20 

step 4
[[2 3 6]
 [1 0 5]
 [4 7 8]]
action= down , depth= 4 , step cost= 7 , total_cost= 27 

step 5
[[2 3 6]
 [1 5 0]
 [4 7 8]]
action= left , depth= 5 , step cost= 5 , total_cost= 32 

step 6
[[2 3 0]
 [1 5 6]
 [4 7 8]]
action= down , depth= 6 , step cost= 6 , total_cost= 38 

step 7
[[2 0 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 7 , step cost= 3 , total_cost= 41 

step 8
[[0 2 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 8 , step cost= 2 , total_cost= 43 

step 9
[[1 2 3]
 [0 5 6]
 [4 7 8]]
action= up , depth= 9 , step cost= 1 , total_cost= 44 

step 10
[[1 2 3]
 [4 5 6]
 [0 7 8]]
action= up , depth= 10 , step cost= 4

'Search Completed'

In [46]:
# perform DFS one level deeper at a time, using looping stack
root_node.iterative_deepening_DFS(goal_state)

what is the dept limit: 15
step 0
[[2 3 6]
 [1 0 7]
 [4 8 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 3 6]
 [1 7 0]
 [4 8 5]]
action= left , depth= 1 , step cost= 7 , total_cost= 7 

step 2
[[2 3 6]
 [1 7 5]
 [4 8 0]]
action= up , depth= 2 , step cost= 5 , total_cost= 12 

step 3
[[2 3 6]
 [1 7 5]
 [4 0 8]]
action= right , depth= 3 , step cost= 8 , total_cost= 20 

step 4
[[2 3 6]
 [1 0 5]
 [4 7 8]]
action= down , depth= 4 , step cost= 7 , total_cost= 27 

step 5
[[2 3 6]
 [1 5 0]
 [4 7 8]]
action= left , depth= 5 , step cost= 5 , total_cost= 32 

step 6
[[2 3 0]
 [1 5 6]
 [4 7 8]]
action= down , depth= 6 , step cost= 6 , total_cost= 38 

step 7
[[2 0 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 7 , step cost= 3 , total_cost= 41 

step 8
[[0 2 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 8 , step cost= 2 , total_cost= 43 

step 9
[[1 2 3]
 [0 5 6]
 [4 7 8]]
action= up , depth= 9 , step cost= 1 , total_cost= 44 

step 10
[[1 2 3]
 [4 5 6]
 [0 7 8]]
action= up

'Search Complete'

In [47]:
# search based on manhattan distance as the heuristic cost
root_node.greedy_search(goal_state)

step 0
[[2 3 6]
 [1 0 7]
 [4 8 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 3 6]
 [1 7 0]
 [4 8 5]]
action= left , depth= 1 , step cost= 7 , total_cost= 20 

step 2
[[2 3 0]
 [1 7 6]
 [4 8 5]]
action= down , depth= 2 , step cost= 6 , total_cost= 25 

step 3
[[2 0 3]
 [1 7 6]
 [4 8 5]]
action= right , depth= 3 , step cost= 3 , total_cost= 27 

step 4
[[0 2 3]
 [1 7 6]
 [4 8 5]]
action= right , depth= 4 , step cost= 2 , total_cost= 28 

step 5
[[1 2 3]
 [0 7 6]
 [4 8 5]]
action= up , depth= 5 , step cost= 1 , total_cost= 28 

step 6
[[1 2 3]
 [4 7 6]
 [0 8 5]]
action= up , depth= 6 , step cost= 4 , total_cost= 31 

step 7
[[1 2 3]
 [4 7 6]
 [8 0 5]]
action= left , depth= 7 , step cost= 8 , total_cost= 38 

step 8
[[1 2 3]
 [4 0 6]
 [8 7 5]]
action= down , depth= 8 , step cost= 7 , total_cost= 44 

step 9
[[1 2 3]
 [0 4 6]
 [8 7 5]]
action= right , depth= 9 , step cost= 4 , total_cost= 47 

step 10
[[1 2 3]
 [8 4 6]
 [0 7 5]]
action= up , depth= 10 , step cost= 

'Search Complete'

In [48]:
# A*3 search based on path cost+heuristic cost, using priority queue
# No.1 fast
root_node.a_star_search(goal_state,heuristic_function = 'fair_manhattan')

step 0
[[2 3 6]
 [1 0 7]
 [4 8 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 3 6]
 [1 7 0]
 [4 8 5]]
action= left , depth= 1 , step cost= 7 , total_cost= 14 

step 2
[[2 3 6]
 [1 7 5]
 [4 8 0]]
action= up , depth= 2 , step cost= 5 , total_cost= 16 

step 3
[[2 3 6]
 [1 7 5]
 [4 0 8]]
action= right , depth= 3 , step cost= 8 , total_cost= 18 

step 4
[[2 3 6]
 [1 0 5]
 [4 7 8]]
action= down , depth= 4 , step cost= 7 , total_cost= 18 

step 5
[[2 3 6]
 [1 5 0]
 [4 7 8]]
action= left , depth= 5 , step cost= 5 , total_cost= 20 

step 6
[[2 3 0]
 [1 5 6]
 [4 7 8]]
action= down , depth= 6 , step cost= 6 , total_cost= 20 

step 7
[[2 0 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 7 , step cost= 3 , total_cost= 20 

step 8
[[0 2 3]
 [1 5 6]
 [4 7 8]]
action= right , depth= 8 , step cost= 2 , total_cost= 20 

step 9
[[1 2 3]
 [0 5 6]
 [4 7 8]]
action= up , depth= 9 , step cost= 1 , total_cost= 20 

step 10
[[1 2 3]
 [4 5 6]
 [0 7 8]]
action= up , depth= 10 , step cost= 

'Search Complete'

In [50]:
# search as far as possible along each branch before backtracking, using stack
root_node.depth_first_search(goal_state)

Streaming output truncated to the last 5000 lines.
step 100848
[[0 6 3]
 [1 8 4]
 [5 7 2]]
action= down , depth= 100848 , step cost= 1 , total_cost= 454005 

step 100849
[[6 0 3]
 [1 8 4]
 [5 7 2]]
action= left , depth= 100849 , step cost= 6 , total_cost= 454011 

step 100850
[[6 3 0]
 [1 8 4]
 [5 7 2]]
action= left , depth= 100850 , step cost= 3 , total_cost= 454014 

step 100851
[[6 3 4]
 [1 8 0]
 [5 7 2]]
action= up , depth= 100851 , step cost= 4 , total_cost= 454018 

step 100852
[[6 3 4]
 [1 0 8]
 [5 7 2]]
action= right , depth= 100852 , step cost= 8 , total_cost= 454026 

step 100853
[[6 3 4]
 [0 1 8]
 [5 7 2]]
action= right , depth= 100853 , step cost= 1 , total_cost= 454027 

step 100854
[[0 3 4]
 [6 1 8]
 [5 7 2]]
action= down , depth= 100854 , step cost= 6 , total_cost= 454033 

step 100855
[[3 0 4]
 [6 1 8]
 [5 7 2]]
action= left , depth= 100855 , step cost= 3 , total_cost= 454036 

step 100856
[[3 4 0]
 [6 1 8]
 [5 7 2]]
action= left , depth= 100856 , step cost= 4 , total_c

'Search Complete'